In [160]:
import requests
from bs4 import BeautifulSoup

res = requests.get('http://news.sina.com.cn/china/')
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')

for news in soup.select('.news-item'):
    if len(news.select('h2')) > 0:
        time = news.select('.time')[0].text
        h2 = news.select('h2')[0].text
        a = news.select('a')[0]['href']
        print(time, h2, a)

12月22日 14:44 开国中将郑维山之女郑明民在京因车祸不幸逝世 http://news.sina.com.cn/c/nd/2017-12-22/doc-ifypvuqf0870575.shtml
12月22日 14:35 调查显示：台湾人最在乎钱 对爱情无感创10年新低 http://news.sina.com.cn/c/gat/2017-12-22/doc-ifypvuqf0840234.shtml
12月22日 14:22 合肥天然气用量超红色预警 遇极端天气将优先民用 http://news.sina.com.cn/c/nd/2017-12-22/doc-ifypxrpp3343116.shtml
12月22日 14:20 台学者：“文化台独”欺骗性更大 蔡当局手法恶劣 http://news.sina.com.cn/c/gat/2017-12-22/doc-ifypvuqf0783683.shtml
12月22日 14:12 九旬抗战老兵段开乾辞世 曾冒死穿越日军封锁线 http://news.sina.com.cn/c/nd/2017-12-22/doc-ifypxmsq9339219.shtml
12月22日 14:07 加息火种传到亚洲央行坐不住 未来房贷会变高吗？ http://news.sina.com.cn/o/2017-12-22/doc-ifypxmsq9336083.shtml
12月22日 14:04 “茫”成台年度代表字:蔡当局瞎“忙”老百姓迷茫 http://news.sina.com.cn/c/gat/2017-12-22/doc-ifypxrpp3339312.shtml
12月22日 13:46 吉林大兴沟林区30年来首次发现野生东北虎踪迹 http://news.sina.com.cn/c/nd/2017-12-22/doc-ifypxmsq9328733.shtml
12月22日 13:42 刘培任团武汉市委兼职副书记 曾割皮救父感动全国 http://news.sina.com.cn/c/nd/2017-12-22/doc-ifypxmsq9327454.shtml
12月22日 13:36 9成台民众希望“死得像个人” 赞成安乐死合法化 http://news.sina.com.cn/c/gat/2017-12-2

In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
from datetime import datetime

'''
def getCommentContent(newsurl):
    newsid = re.search('doc-i(.*).shtml', newsurl).group(1)
    commentURL = 'http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=sh&newsid=comos-{}\
&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1'
    comments = requests.get(commentURL.format(newsid))
    jd = json.loads(comments.text)
    return [cont['content'] for cont in jd['result']['cmntlist']]
'''

def getCommentNumber(newsurl):
    newsid = re.search('doc-i(.*).shtml', newsurl).group(1)
    commentURL = 'http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=sh&newsid=comos-{}\
&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1'
    comments = requests.get(commentURL.format(newsid))
    jd = json.loads(comments.text)
    return jd['result']['count']['total']

def getNewsDetail(newsurl):
    result = {}
    res = requests.get(newsurl)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    result['title'] = soup.select('.article p')[0].text.strip().lstrip('原标题：')
    result['news_source'] = soup.select('a')[0]['href']
    time_source = soup.select('.date')[0].text
    result['date_time'] = datetime.strptime(time_source, '%Y年%m月%d日 %H:%M').strftime("%Y-%m-%d %H:%M")
    result['article'] = ' '.join([p.text.strip() for p in soup.select('.article p')[:-1]])
    result['editor'] = soup.select('.show_author')[0].text.lstrip('责任编辑：').strip()
    #result['comment'] = getCommentContent(newsurl)
    result['comment_number'] = getCommentNumber(newsurl)
    return result
    
getNewsDetail('http://news.sina.com.cn/c/nd/2017-12-22/doc-ifypvuqe9719332.shtml')

{'article': '原标题：一年250余次密集调控后 明年楼市或迎新一轮调整期 据每日经济新闻报道，中原地产研究中心统计数据显示，截至12月，年内全国房地产调控政策发布将近110个城市与部门（县级以上），发布的调控政策次数多达250次以上，以手术刀式的精准调控，针对市场变化调整调控内容与加码力度。而《中国住房发展报告》指出，随着本轮楼市回升小周期接近尾声，未来一年将迎来新一轮调整期和降温期。同时，中指院指出，总体来看，本轮周期将持续5年或更长时间，预计2018年市场将出现下行。',
 'comment_number': 3,
 'date_time': '2017-12-22 09:53',
 'editor': '张玉',
 'news_source': 'http://www.sina.com.cn/',
 'title': '一年250余次密集调控后 明年楼市或迎新一轮调整期'}

In [2]:
import requests
import json

def parseListLinks(url):
    news_detail = []
    res = requests.get(url)
    jd = json.loads(res.text.lstrip('  newsloadercallback(').rstrip(');'))
    for ent in jd['result']['data']:
        news_detail.append(getNewsDetail(ent['url']))
    return news_detail

url = 'http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&\
level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&callback=newsloadercallback&_=1513930178035'
news_total = []
for i in range(1, 3):
    news_url = url.format(i)
    news_ary = parseListLinks(news_url)
    news_total.extend(news_ary)
len(news_total)

44

In [3]:
import pandas

df = pandas.DataFrame(news_total)
df

,article,comment_number,date_time,editor,news_source,title
0,原标题：60后中候补的最新任命 来源：长安街知事 卸任唐山市委书记后，河北省委常委焦彦龙的去...,0,2017-12-22 19:00,霍宇昂,http://www.sina.com.cn/,60后中候补的最新任命
1,[国家发改委：坚持在保障民生基本需求前提下积极推进北方地区清洁供暖工程]在22日召开的全国发...,0,2017-12-22 18:59,张岩,http://www.sina.com.cn/,[国家发改委：坚持在保障民生基本需求前提下积极推进北方地区清洁供暖工程]在22日召开的全国发...
2,原标题：蔡当局搜查新党青年惹众怒 “拔菜总部”高喊“下台” 海外网12月22日电 12月22...,0,2017-12-22 18:32,张岩,http://www.sina.com.cn/,蔡当局搜查新党青年惹众怒 “拔菜总部”高喊“下台”
3,原标题：[荐读]被外交部发言人华春莹的大笑圈粉了！网友：可爱又迷人 一直以来，我外交部发言人...,1,2017-12-22 18:41,霍宇昂,http://www.sina.com.cn/,[荐读]被外交部发言人华春莹的大笑圈粉了！网友：可爱又迷人
4,原标题 强行加戏刷存在感？绿营名嘴：新党青年拿大陆500万，要建军事组织… 19日，新党青年...,16,2017-12-22 18:34,张岩,http://www.sina.com.cn/,强行加戏刷存在感？绿营名嘴：新党青年拿大陆500万，要建军事组织…
5,原标题：中国药品出口腹背受敌：高端药难敌欧美，低端药做不过印度 西药制剂全球竞争如同赛跑，中...,0,2017-12-22 18:21,霍宇昂,http://www.sina.com.cn/,中国药品出口腹背受敌：高端药难敌欧美，低端药做不过印度
6,原标题：陕西周至安装路灯一盏5.5万？官方：工作人员表述不准确 红星新闻12月22日消息，今...,0,2017-12-22 18:20,霍宇昂,http://www.sina.com.cn/,陕西周至安装路灯一盏5.5万？官方：工作人员表述不准确
7,原标题：“中国教授炮轰法国”视频在外网火了，然而真相却令国人愤怒 一条中文采访视频被配上完全...,16,2017-12-22 18:12,张岩,http://www.sina.com.cn/,“中国教授炮轰法国”视频在外网火了，然而真相却令国人愤怒
8,原标题：与令计划有关的9个人 12月22日，冬至。 3年前的冬至日，晚上8时，中纪委官网发布...,7,2017-12-22 17:54,霍宇昂,http://www.sina.com.cn/,与令计划有关的9个人
9,原标题： 落马高官的3块手表，拍卖了23.3万 21日，北京法院审判信息网公布了2份北京市第...,2,2017-12-22 17:54,霍宇昂,http://www.sina.com.cn/,落马高官的3块手表，拍卖了23.3万


In [4]:
df.to_excel('news.xlsx')

In [5]:
import sqlite3

with sqlite3.connect('news.sqlite') as db:
    df.to_sql('news_table', con = db, if_exists = 'replace')

In [6]:
import sqlite3
import pandas

with sqlite3.connect('news.sqlite') as db:
    df2 = pandas.read_sql_query('select * from news_table', con = db)
df2

,index,article,comment_number,date_time,editor,news_source,title
0,0,原标题：60后中候补的最新任命 来源：长安街知事 卸任唐山市委书记后，河北省委常委焦彦龙的去...,0,2017-12-22 19:00,霍宇昂,http://www.sina.com.cn/,60后中候补的最新任命
1,1,[国家发改委：坚持在保障民生基本需求前提下积极推进北方地区清洁供暖工程]在22日召开的全国发...,0,2017-12-22 18:59,张岩,http://www.sina.com.cn/,[国家发改委：坚持在保障民生基本需求前提下积极推进北方地区清洁供暖工程]在22日召开的全国发...
2,2,原标题：蔡当局搜查新党青年惹众怒 “拔菜总部”高喊“下台” 海外网12月22日电 12月22...,0,2017-12-22 18:32,张岩,http://www.sina.com.cn/,蔡当局搜查新党青年惹众怒 “拔菜总部”高喊“下台”
3,3,原标题：[荐读]被外交部发言人华春莹的大笑圈粉了！网友：可爱又迷人 一直以来，我外交部发言人...,1,2017-12-22 18:41,霍宇昂,http://www.sina.com.cn/,[荐读]被外交部发言人华春莹的大笑圈粉了！网友：可爱又迷人
4,4,原标题 强行加戏刷存在感？绿营名嘴：新党青年拿大陆500万，要建军事组织… 19日，新党青年...,16,2017-12-22 18:34,张岩,http://www.sina.com.cn/,强行加戏刷存在感？绿营名嘴：新党青年拿大陆500万，要建军事组织…
5,5,原标题：中国药品出口腹背受敌：高端药难敌欧美，低端药做不过印度 西药制剂全球竞争如同赛跑，中...,0,2017-12-22 18:21,霍宇昂,http://www.sina.com.cn/,中国药品出口腹背受敌：高端药难敌欧美，低端药做不过印度
6,6,原标题：陕西周至安装路灯一盏5.5万？官方：工作人员表述不准确 红星新闻12月22日消息，今...,0,2017-12-22 18:20,霍宇昂,http://www.sina.com.cn/,陕西周至安装路灯一盏5.5万？官方：工作人员表述不准确
7,7,原标题：“中国教授炮轰法国”视频在外网火了，然而真相却令国人愤怒 一条中文采访视频被配上完全...,16,2017-12-22 18:12,张岩,http://www.sina.com.cn/,“中国教授炮轰法国”视频在外网火了，然而真相却令国人愤怒
8,8,原标题：与令计划有关的9个人 12月22日，冬至。 3年前的冬至日，晚上8时，中纪委官网发布...,7,2017-12-22 17:54,霍宇昂,http://www.sina.com.cn/,与令计划有关的9个人
9,9,原标题： 落马高官的3块手表，拍卖了23.3万 21日，北京法院审判信息网公布了2份北京市第...,2,2017-12-22 17:54,霍宇昂,http://www.sina.com.cn/,落马高官的3块手表，拍卖了23.3万
